# Anomalib Models
In this notebook, we show how anomalib models could be initialized via Python API. As shown in [README.md](https://github.com/openvinotoolkit/anomalib#training), following models are supported in anomalib:

- [CFlow](anomalib/models/cflow)
- [DFKDE](anomalib/models/dfkde)
- [DFM](anomalib/models/dfm)
- [Draem](anomalib/models/draem)
- [FastFlow](anomalib/models/fastflow)
- [GANomaly](anomalib/models/ganomaly)
- [PADIM](anomalib/models/padim)
- [PatchCore](anomalib/models/patchcore)
- [Reverse Distillation](anomalib/models/reverse_distillation)
- [STFPM](anomalib/models/stfpm)




In [19]:

from functools import partial
from torch.optim import Optimizer
from pathlib import Path
from types import MethodType
from typing import Any
from pytorch_lightning import Trainer, LightningModule
from pytorch_lightning.callbacks import ModelCheckpoint

from anomalib.data.mvtec import MVTec
from anomalib.models.fastflow.lightning_model import Fastflow
from anomalib.models.fastflow.torch_model import FastflowModel
from anomalib.models.patchcore import Patchcore
from anomalib.utils.callbacks import (
    MetricsConfigurationCallback,
    MinMaxNormalizationCallback,
    VisualizerCallback,
)

## Data Module
To train each model end-to-end, we do need to have a dataset. In our [previous notebooks](https://github.com/openvinotoolkit/anomalib/tree/development/notebooks/100_datamodules), we demonstrate how to initialize benchmark and custom datasets. In this tutorial, we will use MVTec AD DataModule. We assume that `datasets` directory is created in the `anomalib` root directory and `MVTec` dataset is located in `datasets` directory. Let's confirm this with the following:

In [2]:
Path.cwd()

PosixPath('/home/sakcay/projects/anomalib/notebooks/200_models')

In [3]:
# Go to the main anomalib root dir
root = Path.cwd().parent.parent
root

PosixPath('/home/sakcay/projects/anomalib')

In [4]:
list((root / "datasets").iterdir())

[PosixPath('/home/sakcay/projects/anomalib/datasets/BTech'),
 PosixPath('/home/sakcay/projects/anomalib/datasets/MVTec'),
 PosixPath('/home/sakcay/projects/anomalib/datasets/bottle'),
 PosixPath('/home/sakcay/projects/anomalib/datasets/README.txt'),
 PosixPath('/home/sakcay/projects/anomalib/datasets/hazelnut_toy')]

Now that we checked we have `datasets` directory and `MVTec` is already located in `datasets`, we could create the datamodule.

In [ ]:
MVTec??

In [7]:
datamodule = MVTec(
    root="../../datasets/MVTec/",
    category="bottle",
    image_size=256,
    train_batch_size=32,
    test_batch_size=32,
    num_workers=8,
    task="segmentation",
)
datamodule.setup()
i, data = next(enumerate(datamodule.test_dataloader()))
data["image"].shape, data["mask"].shape

(torch.Size([32, 3, 256, 256]), torch.Size([32, 256, 256]))

## Models
Now that we have created MVTec datamodule, we could create the models. We could start with `PatchCore` since it is the ranked #1 model on the MVTec AD category on papers with code. 

Each model on anomalib has the following structure:
```
        anomalib/models/<model_name>
        ├── README.md           # Readme file containing description and benchmarks.
        ├── __init__.py         # Model initialization.
        ├── config.yaml         # Stores the model configurations.  
        ├── torch_model.py      # Torch model implementing the basic forward-pass mechanism.
        ├── anomaly_map.py      # [Optional] module generating anomaly heatmaps.      
        ├── lightning_model.py  # Lightning module implementing training mechanism
        └── loss.py             # [Optional] module implementing loss computation. 
```



### Torch Model

In [ ]:
FastflowModel??

In [8]:
torch_model = FastflowModel(input_size=[256, 256], backbone="resnet18", flow_steps=8)

As mentioned, `torch_model` implements the basic forward-pass mechanism of the model for both `train` and `test` phases. In `train` phase, the model returns the training output such as feature maps. During the `test` phase, it returns the anomaly heatmap.

In [10]:
torch_model.training = True
train_output = torch_model(data["image"])
hidden_variables, log_jacobian = train_output
hidden_variables[0].shape

torch.Size([32, 64, 64, 64])

In [11]:
torch_model.training = False
anomaly_map = torch_model(data["image"])
anomaly_map.shape

torch.Size([32, 1, 256, 256])

As shown above, when the `training` is `False`, it means the model is in val/test/inference stage, which produces the `anomaly_map`.

### Lightning Module
The main module for each anomalib models is the `LightningModule` that stores the torch_model as its attribute and sorts out the train-test mechanism. Let's see how the LightningModule of `Fastflow` model is instantiated.

In [ ]:
Fastflow??

In [9]:
model = Fastflow(
    input_size=[256, 256],
    backbone="resnet18",
    flow_steps=8,
)

In [10]:
model.training = True
train_output = model(data["image"])
hidden_variables, log_jacobian = train_output
hidden_variables[0].shape

torch.Size([32, 64, 64, 64])

As can be seen above, the Lightning Module also returns the same output as `torch_model`. This is because it stores `torch_model` as its attribute and uses it in its `forward` method. Therefore, it is possible to call the forward-pass with `model(x)`. For the implementational details, you could refer to [this](https://github.com/openvinotoolkit/anomalib/blob/development/anomalib/models/components/base/anomaly_module.py#L64) link. 

Similar to the `torch_model`, `lightning_module` also produces an `anomaly_map` when the `training` is set to `False`.

In [11]:
model.model.training = False
anomaly_map = model(data["image"])
anomaly_map.shape

torch.Size([32, 1, 256, 256])

## Training
Now that we see how anomalib models are structured, we could try to train a model. In order to properly train the models we need to set number of callbacks such as model saving and early stopping.

Let's create these callbacks.

## Model
Now that we created the datamodule, let's create the model. We could use Patchcore model as it currently achieves the highest performance on MVTec dataset.

In [29]:
def configure_optimizers(lightning_module: LightningModule, optimizer: Optimizer) -> Any:
    """Override to customize the :meth:`~pytorch_lightning.core.lightning.LightningModule.configure_optimizers`
    method.

    Args:
        lightning_module: A reference to the model.
        optimizer: The optimizer.
        lr_scheduler: The learning rate scheduler (if used).
    """
    return optimizer

In [30]:
from torch.optim.adam import Adam
optimizer = Adam(params=model.parameters(), lr=0.001, betas=(0.9, 0.999), weight_decay=1e-5)

fn = partial(configure_optimizers, optimizer=optimizer)
# update_wrapper(fn, configure_optimizers)  # necessary for `is_overridden`

model.configure_optimizers = MethodType(fn, model)

In [31]:
fn

functools.partial(<function configure_optimizers at 0x7ff5ae9b3c10>, optimizer=Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.001
    weight_decay: 1e-05
))

In [32]:
model.configure_optimizers()

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.001
    weight_decay: 1e-05
)

In [ ]:
model = Patchcore(
    input_size=(224, 224),
    backbone="resnet18",
    layers=["layer3"],
)

In [17]:
callbacks = [
    ModelCheckpoint(),
    MetricsConfigurationCallback(adaptive_threshold=True),
    MinMaxNormalizationCallback(),
    VisualizerCallback(task="segmentation", log_images_to=["local"]),
]

The final component that we need to train a model is `pytorch_lightning` `Trainer` object, which handles train/test/predict pipeline. Let's create the trainer object to train the model.

In [18]:
trainer = Trainer(
    callbacks=callbacks,
    accelerator="auto", # <"cpu", "gpu", "tpu", "ipu", "hpu", "auto">,
    check_val_every_n_epoch=1, # Don't validate before extracting features.,
    devices=1,
    gpus=1, # Set automatically,
    gradient_clip_val=0,
    limit_predict_batches=1.0,
    limit_test_batches=1.0,
    limit_train_batches=1.0,
    limit_val_batches=1.0,
    log_every_n_steps=50,
    max_epochs=1,
    max_steps=-1,
    num_nodes=1,
    num_sanity_val_steps=0,
    overfit_batches=0.0,
    precision=32,
    reload_dataloaders_every_n_epochs=0,
    track_grad_norm=-1,
    val_check_interval=1.0, # Don't validate before extracting features.                  ,
)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(limit_train_batches=1.0)` was configured so 100% of the batches per epoch will be used..
`Trainer(limit_val_batches=1.0)` was configured so 100% of the batches will be used..
`Trainer(limit_test_batches=1.0)` was configured so 100% of the batches will be used..
`Trainer(limit_predict_batches=1.0)` was configured so 100% of the batches will be used..
`Trainer(val_check_interval=1.0)` was configured so validation will run at the end of the training epoch..


`Trainer` object has number of options that suit all specific needs. For more details, refer to [Lightning Documentation](https://pytorch-lightning.readthedocs.io/en/stable/common/trainer.html) to see how it could be tweaked to your needs.

Let's train the model now.

In [19]:
trainer.fit(datamodule=datamodule, model=model)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]

  | Name                  | Type                     | Params
-------------------------------------------------------------------
0 | image_threshold       | AdaptiveThreshold        | 0     
1 | pixel_threshold       | AdaptiveThreshold        | 0     
2 | training_distribution | AnomalyScoreDistribution | 0     
3 | min_max               | MinMax                   | 0     
4 | model                 | PatchcoreModel           | 11.7 M
5 | image_metrics         | AnomalibMetricCollection | 0     
6 | pixel_metrics         | AnomalibMetricCollection | 0     
-------------------------------------------------------------------
11.7 M    Trainable params
0         Non-trainable params
11.7 M    Total params
46.758    Total estimated model params size (MB)


Epoch 0: 100%|██████████| 10/10 [00:10<00:00,  1.06s/it, loss=nan, v_num=3]


In [20]:
trainer.test(datamodule=datamodule, model=model)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]


Testing DataLoader 0: 100%|██████████| 3/3 [00:02<00:00,  1.20it/s]


[{}]